In [20]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
import serial
import time
import numpy as np
import keras
from keras.preprocessing import image
import cv2
import imutils
import numpy as np

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense

In [21]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('HD/train',target_size = (64, 64),batch_size = 32,
                                                 class_mode = 'binary')
test_set = test_datagen.flow_from_directory('HD/test',
                                             target_size = (64, 64),batch_size = 32,class_mode = 'binary')





Found 5966 images belonging to 6 classes.
Found 572 images belonging to 6 classes.


In [22]:

# Initialising the CNN
classifier = Sequential()
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
# Step 2 - Pooling
classifier.add(BatchNormalization(axis=3))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.25))

# Adding a second convolutional layer
classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(BatchNormalization(axis=3))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.25))
classifier.add(Conv2D(128, (3, 3), activation = 'relu'))
classifier.add(BatchNormalization(axis=3))
classifier.add(Conv2D(128, (3, 3), activation = 'relu'))
classifier.add(BatchNormalization(axis=3))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.25))
# Step 3 - Flattening
classifier.add(Flatten())
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 6, activation = 'softmax'))
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])




In [23]:

# Part 2 - Fitting the CNN to the images

r=classifier.fit(training_set,epochs = 10,validation_data = test_set)

Epoch 1/10
187/187 [==============================] - 58s 297ms/step - loss: 0.2734 - accuracy: 0.9212 - val_loss: 10.1407 - val_accuracy: 0.1696
Epoch 2/10
187/187 [==============================] - 41s 221ms/step - loss: 0.0545 - accuracy: 0.9874 - val_loss: 0.9683 - val_accuracy: 0.7465
Epoch 3/10
187/187 [==============================] - 41s 219ms/step - loss: 0.0384 - accuracy: 0.9864 - val_loss: 0.0227 - val_accuracy: 0.9965
Epoch 4/10
187/187 [==============================] - 41s 220ms/step - loss: 0.0436 - accuracy: 0.9883 - val_loss: 0.0197 - val_accuracy: 0.9965
Epoch 5/10
187/187 [==============================] - 41s 220ms/step - loss: 0.0355 - accuracy: 0.9913 - val_loss: 0.0215 - val_accuracy: 0.9965
Epoch 6/10
187/187 [==============================] - 42s 224ms/step - loss: 0.0310 - accuracy: 0.9920 - val_loss: 0.0766 - val_accuracy: 0.9948
Epoch 7/10
187/187 [==============================] - 41s 221ms/step - loss: 0.0222 - accuracy: 0.9935 - val_loss: 0.0504 - val_a

In [201]:
import serial
import time
def write_read(x):
    data =  arduino.write(bytes(x, 'utf-8'))
    return data


In [202]:
arduino = serial.Serial(port='COM4', baudrate=9600, timeout=.5)

In [203]:
global filename
global classifier


bg = None
playcount = 0
names = ['b', 'f', 'l', 'n', 'r', 's']
bgModel = cv2.createBackgroundSubtractorMOG2(0, 50)

In [204]:
def run_avg(image, aWeight):
    global bg
    if bg is None:
        bg = image.copy().astype("float")
        return
    cv2.accumulateWeighted(image, bg, aWeight)

def segment(image, threshold=25):
    global bg
    diff = cv2.absdiff(bg.astype("uint8"), image)
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]
    ( cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts) == 0:
        return
    else:
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)


def webcamPredict():
    
    oldresult = 'none'
    nval=0
    count = 0
    fgbg2 = cv2.createBackgroundSubtractorKNN(); 
    aWeight = 0.5
    camera = cv2.VideoCapture(0)
    top, right, bottom, left = 10, 350, 325, 690
    num_frames = 0
    while(True):
        (grabbed, frame) = camera.read()
        frame = imutils.resize(frame, width=700)
        frame = cv2.flip(frame, 1)
        clone = frame.copy()
        (height, width) = frame.shape[:2]
        roi = frame[top:bottom, right:left]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (41, 41), 0)
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:
            temp = gray
            hand = segment(gray)
            if hand is not None:
                (thresholded, segmented) = hand
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                #cv2.imwrite("test.jpg",temp)
                #cv2.imshow("Thesholded", temp)
                #ret, thresh = cv2.threshold(temp, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                #thresh = cv2.resize(thresh, (64, 64))
                #thresh = np.array(thresh)
                #img = np.stack((thresh,)*3, axis=-1)
                roi = frame[top:bottom, right:left]
                roi = fgbg2.apply(roi); 
                cv2.imwrite("test.jpg",roi)
                #cv2.imwrite("newDataset/Fist/"+str(count)+".png",roi)
                #count = count + 1
                #print(count)
                
                img = cv2.imread("test.jpg")
                img = cv2.resize(img, (64, 64))
                img = img.reshape(1, 64, 64, 3)
                img = np.array(img, dtype='float32')
                img /= 255
                
                predict = classifier.predict(img)
                value = np.amax(predict)
                cl = np.argmax(predict)
                result = names[np.argmax(predict)]
                if value >= 0.99:
                    if(result=='n' and nval<1):
                        print(str(value)+" "+str(result))
                        value = write_read(result)
                        nval=1
                    elif(result!='n'): #will make the motion continues even if the hand is shown once
                        print(str(value)+" "+str(result))
                        value = write_read(result)
                        if oldresult != result:
                            oldresult=result
                            print(str(value)+" "+str(oldresult))
                            value = write_read(oldresult)
                    elif(result=='n' and nval>=1): 
                        print(str(value)+" "+str(oldresult))
                        value = write_read(oldresult)
                else:
                    print("Low accuracy")
                    #print(str(value)+" "+str(result))
                    #value = write_read(result)                                  
                    #oldresult = 'Gesture Recognize as : '+str(result)
                    cv2.putText(clone, 'Gesture Recognize as : '+str(result), (10, 25),  cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 255, 255), 2)
                    #if oldresult != result:
                        #print("")
                #else:
                    #cv2.putText(clone, oldresult, (10, 25),  cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 255, 255), 2)
                cv2.imshow("video frame", roi)
            else:
                cv2.putText(clone, 'No Motion', (10, 25),  cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 255, 255), 2)
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)
        num_frames += 1
        cv2.imshow("Video Feed", clone)
        keypress = cv2.waitKey(1) & 0xFF
        if keypress == ord("q"):
            break
    camera.release()
    cv2.destroyAllWindows()    
    


In [205]:
webcamPredict()

1/1 [==============================] - 0s 20ms/step
Low accuracy
1/1 [==============================] - 0s 19ms/step
Low accuracy
1/1 [==============================] - 0s 18ms/step
Low accuracy
1/1 [==============================] - 0s 37ms/step
Low accuracy
1/1 [==============================] - 0s 19ms/step
0.99990416 n
1/1 [==============================] - 0s 28ms/step
0.9999089 none
1/1 [==============================] - 0s 21ms/step
0.9999132 none
1/1 [==============================] - 0s 24ms/step
0.99989414 none
1/1 [==============================] - 0s 18ms/step
0.99989414 none
1/1 [==============================] - 0s 18ms/step
0.9998958 none
1/1 [==============================] - 0s 21ms/step
0.9999014 none
1/1 [==============================] - 0s 20ms/step
0.99990165 none
1/1 [==============================] - 0s 19ms/step
0.9999999 none
1/1 [==============================] - 0s 21ms/step
0.9999 none
1/1 [==============================] - 0s 20ms/step
0.99947256 none
1/1 